In [ ]:
!pip3 install --upgrade pip
!pip3 install pandas
!pip3 install ipython-sql prettytable 
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install numpy
!pip3 install scipy
!pip3 install scikit-learn

!pip3 install ipython-sql


import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

con = sqlite3.connect('data.db')

cursor = con.cursor()




In [ ]:
# import chicago socioeconomic census data 
chicago_census_data = pd.read_csv ('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
chicago_census_data.to_sql("CHICAGO_CENSUS_DATA", con, if_exists='replace', index=False, method="multi")


In [ ]:
#chicago public school data
chicago_public_schools = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
chicago_public_schools.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False)


In [ ]:
# chicago crime data
chicago_crime_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
chicago_crime_data.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False)



In [ ]:
# loading sql magic module to use % commands
%load_ext sql

# creating a connection between sql magic and 'FinalDB.db' database
%sql sqlite:///data.db

# **SQL Data Querying**

## **Basic Descriptive Analysis**

In [ ]:
%sql SELECT \
    MIN(PER_CAPITA_INCOME) as MIN_Income, \
    MAX(PER_CAPITA_INCOME) as MAX_Income, \
    AVG(PER_CAPITA_INCOME) as AVG_Income, \
    MIN(HARDSHIP_INDEX) as MIN_hardship, \
    MAX(HARDSHIP_INDEX) as MAX_hardship, \
    AVG(HARDSHIP_INDEX) as AVG_hardship \
    FROM CHICAGO_CENSUS_DATA;

In [ ]:
%sql PRAGMA table_info(CHICAGOPUBLICSCHOOLS);

## **Most Crime Prone Community Names & Area Number**

In [ ]:
%sql SELECT CE.COMMUNITY_AREA_NAME, CR.COMMUNITY_AREA_NUMBER, COUNT(*) \
AS crime_count FROM CHICAGO_CRIME_DATA AS CR JOIN CHICAGO_CENSUS_DATA \
AS CE ON CR.COMMUNITY_AREA_NUMBER = CE.COMMUNITY_AREA_NUMBER GROUP BY \
CR.COMMUNITY_AREA_NUMBER, CE.COMMUNITY_AREA_NAME ORDER BY crime_count DESC;


##  **Finding the community with the highest hardship index**

In [ ]:
%sql SELECT COMMUNITY_AREA_NAME, hardship_index FROM CHICAGO_CENSUS_DATA \
ORDER BY hardship_index DESC LIMIT 10;



##  **Correlation between income and hardship[*]**

In [ ]:
%sql SELECT \
    (SUM(PER_CAPITA_INCOME * HARDSHIP_INDEX) - SUM(PER_CAPITA_INCOME) * SUM(HARDSHIP_INDEX) / COUNT(*)) / \
    (SQRT(SUM(PER_CAPITA_INCOME * PER_CAPITA_INCOME) - SUM(PER_CAPITA_INCOME) * SUM(PER_CAPITA_INCOME) / COUNT(*)) * \
     SQRT(SUM(HARDSHIP_INDEX * HARDSHIP_INDEX) - SUM(HARDSHIP_INDEX) * SUM(HARDSHIP_INDEX) / COUNT(*))) as correlation \
FROM CHICAGO_CENSUS_DATA;


##  **Income disparity analysis[*]**

In [ ]:
%sql WITH IncomeQuartiles AS ( \
    SELECT PER_CAPITA_INCOME, NTILE(4) OVER (ORDER BY PER_CAPITA_INCOME) as quartile FROM Chicago_Census_Data) \
SELECT quartile, \
    MIN(PER_CAPITA_INCOME) as min_income, \
    MAX(PER_CAPITA_INCOME) as max_income, \
    MAX(PER_CAPITA_INCOME) - MIN(PER_CAPITA_INCOME) as range, \
    MAX(PER_CAPITA_INCOME) / MIN(PER_CAPITA_INCOME) as ratio \
FROM IncomeQuartiles \
GROUP BY quartile \
ORDER BY quartile;


## **Per Capita Income in Community by *Highest* Percentage of Households Below Poverty Level**

In [ ]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY, PER_CAPITA_INCOME \
FROM CHICAGO_CENSUS_DATA \
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY \
DESC LIMIT 10 



## **Per Capita Income in Community by *Lowest* Percentage of Households Below Poverty Level**

In [ ]:
# Per capita income in community by lowest percentage of households below poverty level
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY, PER_CAPITA_INCOME \
FROM CHICAGO_CENSUS_DATA \
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY ASC \
LIMIT 10 


## **Total College Enrollment from Community Area**

In [ ]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS_DATA \
   group by Community_Area_Name 

## **Community Areas with the least total College Enrollment**

In [ ]:
%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from CHICAGO_PUBLIC_SCHOOLS_DATA \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   LIMIT 10

## **School Safety Scores in Descending Order**

In [ ]:
%sql SELECT name_of_school, safety_score \
FROM CHICAGO_PUBLIC_SCHOOLS_DATA  where safety_score !='None' \
ORDER BY safety_score LIMIT 10